In [ ]:
%pwd
%cd ../..

In [ ]:
import argparse
import sys
from datetime import datetime
from pathlib import Path

import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from dotenv import load_dotenv, find_dotenv
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TerminateOnNaN
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from models import build_simple_conv_net

In [ ]:
BASE_DIR = Path().resolve()
LOGS_DIR = BASE_DIR.joinpath('logs', f'{datetime.now().strftime("%m-%d-%Y_%H:%M:%S")}')

WEIGHTS_OUTPUT_PATH = LOGS_DIR.joinpath('ep{epoch:03d}-loss{loss:.4f}-val_loss{val_loss:.4f}.h5')
MODEL_OUTPUT_PATH = LOGS_DIR.joinpath('trained_model.h5')

ENV_FILE = find_dotenv()
if ENV_FILE:
    load_dotenv(ENV_FILE)

In [ ]:
input_shape = np.asarray([300, 300, 3])
checkpoint = ModelCheckpoint(str(WEIGHTS_OUTPUT_PATH),
                             monitor='val_loss',
                             verbose=1,
                             save_weights_only=True,
                             save_best_only=True)
logging = TensorBoard(log_dir=str(LOGS_DIR))
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=50, verbose=1)
terminate_on_nan = TerminateOnNaN()
callbacks = [logging, checkpoint, early_stopping, terminate_on_nan]

optimizer = Adam()
model = build_simple_conv_net(input_shape)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', tfa.metrics.F1Score(num_classes=2,average="micro")])
epochs = 100

In [ ]:
train_datagen = ImageDataGenerator(rescale=1 / 255)
val_datagen = ImageDataGenerator(rescale=1 / 255)

train_generator = train_datagen.flow_from_directory(
    '/var/data/Downloads/archive/chest_xray/chest_xray/train/',
    target_size=input_shape[:2],
    batch_size=16,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    '/var/data/Downloads/archive/chest_xray/chest_xray/val/',
    target_size=input_shape[:2],
    batch_size=16,
    class_mode='binary'
)


In [ ]:
history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator,
        callbacks=callbacks
    )

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1/255)

test_generator = test_datagen.flow_from_directory(
    '/var/data/Downloads/archive/chest_xray/chest_xray/test/',
    target_size=input_shape[:2],
    batch_size = 156, 
    class_mode = 'binary'
)


eval_result = model.evaluate(test_generator)

In [ ]:
print(f'test loss: {eval_result[0]}')
print(f'test accuracy: {eval_result[1]}')

In [ ]:
model.save(MODEL_OUTPUT_PATH)